In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']=str(0)

import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import csv
from keras.models import Sequential, Model
from keras.layers import Conv2D, ConvLSTM2D, Dense, MaxPooling2D, Dropout, Flatten, Reshape, merge, Input
from keras.optimizers import Adam

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('image_dir', '/notebooks/udacity/car_data/data/IMG/', 'Simulator Image data')
flags.DEFINE_string('data_path', '/notebooks/udacity/car_data/data/driving_log.csv', 'Simulator CSV')
flags.DEFINE_float('learn_rate', 0.0001, 'Trainign learning rate')

print ('Init completed')

with open(FLAGS.data_path, 'r') as f:
    reader = csv.reader(f)
    csv = np.array([row for row in reader][1:])
# center image, left image, right image
# steering (-0.8, 0.8)
# throttle (0, 1)
# brake (0, 1)
# speed (0, 9.8)

# Process single image
def proc_img(img): # input is 160x320x3
    img = img[59:138:2, 0:-1:2, :] # select vertical region and take each second pixel to reduce image dimensions
    img = (img / 127.5) - 1.0 # normalize colors from 0-255 to -1.0 to 1.0
    return img # return 40x160x3 image

# Read image names and remove IMG/ prefix
image_names_left = np.array(csv[:,1])
image_names_right = np.array(csv[:,2])
image_names_full = np.concatenate((image_names_left, image_names_right))
# read steering data and apply adjustment for left / right images
y_data = np.array(csv[:,3], dtype=float)
y_data_left = y_data+0.08
y_data_right = y_data-0.08
y_data_full = np.concatenate((y_data_left, y_data_right))
print ('CSV loaded')


from get_images import get_images

datadirs=['/notebooks/udacity/new_training/map1_backward/',
                 '/notebooks/udacity/new_training/map1_forward/',
                 '/notebooks/udacity/new_training/map1_recovery_backward/',
                 '/notebooks/udacity/new_training/map1_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_forward/',
                 '/notebooks/udacity/new_training/map2_backward/',
                 '/notebooks/udacity/new_training/map2_recovery_forward/',
                 '/notebooks/udacity/new_training/map2_recovery_backward/',
                   '/notebooks/udacity/new_training/map1_error_correction/',
                   '/notebooks/udacity/new_training/map2_error_correction/'
         ]

images=get_images(datadirs,0.08)
images=images[images.inverse==1]
images=images[images.side!=0]
image_names_full, y_data_full = images.img.values, images.real.values

# Random sort for data and split test and validation sets
def newRandomTestValidationSplit(X, y):
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.01, random_state=111)
    return X_tr, X_val, y_tr, y_val

# Batch generator for training data
def generate_image_batch_tr(names, y_data, batch_size = 32):
    total_items = len(names)
    curr_item = 0
    while (True):
        image_data = np.zeros((batch_size,40,160, 3),dtype=float)
        steering_data = np.zeros((batch_size),dtype=float)
        for j in range(batch_size):
            image_name = names[curr_item]
            image = mpimg.imread(image_name)
            image_data[j] = proc_img(image)
            steering_data[j] = y_data[curr_item]
            curr_item = (curr_item+1)%total_items
        yield image_data, steering_data

# Batch generator for validation data (in this implementation same as for training data)
generate_image_batch=generate_image_batch_tr


Using TensorFlow backend.


Init completed
CSV loaded


/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
from keras.layers import Cropping2D, Lambda


# ----------------------
# Model - ideas from VG type network
inp = Input(shape=(160,320,3))


x=Cropping2D(cropping=((60,20), (0,0)))(inp)

x=Lambda(lambda x: (x / 255.0) - 0.5)(x)

x= MaxPooling2D((4,4))(x)

# First convolution is for model to determine the 'best' colorspace weights
#x = Conv2D(3, 1, 1, border_mode='same', activation='relu')(x)
# Reduce dimensions
x = MaxPooling2D((2,2))(x) #20x80

# First convolution layer
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x)
x1 = Conv2D(32, 3, 3, border_mode='same', activation='relu')(x1)
x1 = MaxPooling2D((2,2))(x1) #10x40
x1 = Dropout(0.5)(x1)
flat1 = Flatten()(x1) # Used for the merge before first fully connected layer

# Second convolution layer
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x1)
x2 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x2 = MaxPooling2D((2,2))(x2) #5x20
x2 = Dropout(0.5)(x2)
flat2 = Flatten()(x2) # Used for the merge before first fully connected layer

# Second convolution layer
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x2)
x3 = Conv2D(64, 3, 3, border_mode='same', activation='relu')(x3)
x3 = MaxPooling2D((2,2))(x3) #2x10
x3 = Dropout(0.5)(x3)
flat3 = Flatten()(x3) # Used for the merge before first fully connected layer

# Merge the flattened ouputs after each convolution layer
#x4 = merge([flat1, flat2, flat3], mode='concat')
x4=flat3
# Fully connected layers
x5 = Dense(512, activation='relu')(x4)
x6 = Dense(128, activation='relu')(x5)
x7 = Dense(16, activation='relu')(x6)
out = Dense(1, activation='linear')(x7)

model = Model(input=inp, output=out)
model.summary()



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 160, 320, 3)   0                                            
____________________________________________________________________________________________________
cropping2d_2 (Cropping2D)        (None, 80, 320, 3)    0           input_2[0][0]                    
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 80, 320, 3)    0           cropping2d_2[0][0]               
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 20, 80, 3)     0           lambda_2[0][0]                   
___________________________________________________________________________________________

In [6]:
from sklearn.utils import shuffle
import cv2

def generate_image_batch_tr(names, y, batch_size = 32, preprocessing = lambda x:x, inverse=None):
    total_items = len(names)
    curr_item = 0
    if not inverse:
        inverse = np.ones(total_items, dtype=np.uint8)
        
    while (True):
        images = []
        angles = np.zeros((batch_size),dtype=float)
        for j in range(batch_size):
            now=curr_item%total_items
            
            image = cv2.imread(names[now])
            angle=y[now]
            if inverse[now]==-1:
                image=np.fliplr(image)
                angle*=-1
                
            images.append(preprocessing(image))
            angles[j] = y[now]
            
            curr_item +=1
        
        if curr_item>total_items:
            curr_item=curr_item%total_items
            names, y, inverse = shuffle(names, y, inverse)
        
        yield np.array(images), angles

In [ ]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_images_gen2_vgg_v2.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=FLAGS.learn_rate), metrics=['mean_squared_error'])


print ('Split data')
X_tr_names, X_val_names, y_tr, y_val = newRandomTestValidationSplit(image_names_full, y_data_full)

print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(generate_image_batch_tr(X_tr_names, y_tr, 64),samples_per_epoch=len(X_tr_names),
                              nb_epoch=100,
                              validation_data=generate_image_batch_tr(X_val_names, y_val, 32),
                              nb_val_samples=len(y_val),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100
90944/90953 [============================>.] - ETA: 0s - loss: 0.0815 - mean_squared_error: 0.0815  

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_mean_squared_error improved from inf to 0.06610, saving model to model_test_images_gen2_vgg_v2.h5
91008/90953 [==============================] - 80s - loss: 0.0815 - mean_squared_error: 0.0815 - val_loss: 0.0661 - val_mean_squared_error: 0.0661
Epoch 2/100
91008/90953 [==============================] - 80s - loss: 0.0657 - mean_squared_error: 0.0657 - val_loss: 0.0621 - val_mean_squared_error: 0.0621
Epoch 3/100
91008/90953 [==============================] - 80s - loss: 0.0624 - mean_squared_error: 0.0624 - val_loss: 0.0574 - val_mean_squared_error: 0.0574
Epoch 4/100
91008/90953 [==============================] - 81s - loss: 0.0607 - mean_squared_error: 0.0607 - val_loss: 0.0607 - val_mean_squared_error: 0.0607
Epoch 5/100
91008/90953 [==============================] - 80s - loss: 0.0597 - mean_squared_error: 0.0597 - val_loss: 0.0583 - val_mean_squared_error: 0.0583
Epoch 6/100
91008/90953 [==============================] - 81s - loss: 0.0586 - mean_squared_error: 0.

In [4]:
from keras.callbacks import History,TensorBoard, EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint("model_test_images_gen2_vgg_v2.h5" , monitor='val_mean_squared_error', verbose=1,
                          save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_mean_squared_error',\
                           min_delta=0.001, patience=3,
                            verbose=1, mode='min')

# Compile, train and save
#model.compile(optimizer=Adam(lr=FLAGS.learn_rate), loss='mse')


model.compile(loss='mse', optimizer=Adam(lr=FLAGS.learn_rate), metrics=['mean_squared_error'])


print ('Split data')
X_tr_names, X_val_names, y_tr, y_val = newRandomTestValidationSplit(image_names_full, y_data_full)

print ('Start training')
# Training and validation inputs are fed from generators
# Number of samples based on data_set size and adjusted to fit batch size
history = model.fit_generator(generate_image_batch_tr(X_tr_names, y_tr, 64),samples_per_epoch=len(X_tr_names),
                              nb_epoch=100,
                              validation_data=generate_image_batch_tr(X_val_names, y_val, 32),
                              nb_val_samples=len(y_val),  callbacks=[checkpoint, early_stop])

Split data
Start training
Epoch 1/100
90944/90953 [============================>.] - ETA: 0s - loss: 0.0760 - mean_squared_error: 0.0760   

/root/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


Epoch 00000: val_mean_squared_error improved from inf to 0.07365, saving model to model_test_images_gen2_vgg_v2.h5
91008/90953 [==============================] - 108s - loss: 0.0760 - mean_squared_error: 0.0760 - val_loss: 0.0737 - val_mean_squared_error: 0.0737
Epoch 2/100
90240/90953 [============================>.] - ETA: 0s - loss: 0.0653 - mean_squared_error: 0.0653  

KeyboardInterrupt: 

In [22]:
next(q)

StopIteration: 